In this script I'll build the ANN for a regresion

In [2]:
# Importing Libraries
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras import regularizers
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
# from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seanborn as sns
from sklearn.preprocessing import StandardScaler


# import keras_tuner
# import time
# import h5py

ModuleNotFoundError: No module named 'numpy'

In [ ]:
import pandas as pd 